In [80]:
import pandas as pd

import networkx as nx
from networkx.algorithms import bipartite
from networkx.readwrite import json_graph
import json
import community

In [106]:
data = pd.read_csv('workshop_survey_data.csv')
data.columns

Index(['Timestamp', 'first_name', 'twitter', 'department', 'university',
       'place_uni', 'year', 'year_uni', 'today', 'research', 'conference',
       'experience_in_DH', 'DH_methods', 'rationale', 'DH_means',
       'hogwarts_house', 'game_of_thrones', 'keanu'],
      dtype='object')

In [12]:
# data = data[data.first_name != 'Keanu']

In [107]:
data.fillna('', inplace=True)

In [14]:
data

,Timestamp,first_name,twitter,department,university,place_uni,year,year_uni,today,research,conference,experience_in_DH,DH_methods,rationale,DH_means,hogwarts_house,game_of_thrones,keanu
0,5/24/2018 11:39:18,Zoe,@Zoe_LeBlanc,"History, Scholars' Lab","Vanderbilt University, University of Virginia","Nashville, TN",6+,2012-01-01,2018-06-01,"International History, Media History, Text Min...","AHA, SHAFR, DH",I am teaching this workshop 🙃,"Networks, Text Mining, Mapping, Data Visualiza...",Hope to learn about everyone's research intere...,I've been really excited by digital history fo...,Slytherin,Targaryen,Much Ado About Nothing
1,5/24/2018 11:48:35,Keanu,@keanuthings,Comparative Literature,UCLA,"Los Angeles, CA",6+,2012-01-01,2018-06-02,"Dogs, Action Movies, Cars Exploding, Digital H...",MLA,Never heard of it. (This will be a fun week fo...,"Pedagogy, Machine Learning/AI/buzzword tech stuff",I want to learn about the Matrix. Also social ...,"Currently nothing, but who knows about the fut...",Hufflepuff,Red God,Who is Keanu Reeves???
2,5/28/2018 18:46:25,Golnar,@GolnarNemat,History of Art and Architecture,University of Pittsburgh,"Pittsburgh, PA",3,2015-01-01,2018-06-03,"Iranian contemporary and modern art, contempor...",,I've taken a course or attended a workshop,"Networks, Mapping, Data Visualization, Databas...",I am developing a DH project about modern art ...,In addition to make archival data available wi...,Ravenclaw,Dorne,He is overrated
3,5/29/2018 8:39:27,Paul,,English,CMU,"Pittsburgh, PA",interested observer,,2018-06-04,"IA, digital history, interactive fiction, inst...",,I've done some readings and tutorials. Maybe I...,"Networks, Text Mining, Mapping, Data Visualiza...","I know some things, but wonder if I lack a goo...","How we have used digital means to communicate,...",Kitchen staff,"(One of my tests for epic fantasy is, ""do I wa...",Much Ado About Nothing
4,5/29/2018 11:20:15,Richard,,English,CMU,"Pittsburgh, PA",1,2017-01-01,2018-06-05,"Political Discourse, Public Rhetorics, Digital...","RSA, CCCC",I've done some readings and tutorials. Maybe I...,"Networks, Text Mining, Mapping, Data Visualiza...",To learn methods of DH project design + gain f...,"I would take ""digital history"" (broadly) as th...",I don't care about Harry Potter,Red God,Speed
5,5/29/2018 12:15:19,Taylor,@taylormariemal,"English, English as a Second Language",Duquesne University,"Pittsburgh, PA",2,2016-01-01,2018-06-06,"Modernist Poetry, Digital Humanities, Sound St...",MLA,I've taken a course or attended a workshop,"Data Visualization, Databases, Blogging, Pedag...",In hopes of understanding DH a bit better for ...,The ability to store a large sum of informatio...,Ravenclaw,Meh,Matrix Series
6,5/29/2018 14:56:24,Jack,,English,CMU,"Pittsburgh, PA",1,2017-01-01,2018-06-07,"English literature, literary theory, early mod...",MLA,Heard of it but don't really know much,"Networks, Text Mining, Data Visualization, Blo...",Primer on variety of DH tools to inform my fut...,"Network analysis, Data visualization, stylomet...",Slytherin,Red God,Bill & Ted Series
7,5/29/2018 18:34:10,Lily,@lilyibrew,History of Art and Architecture,University of Pittsburgh,"Pittsburgh, PA",3,2015-01-01,2018-06-08,"modern and contemporary photography, aerial ph...",CAA,I've taken a course or attended a workshop,"Networks, Mapping, Website Creation",,,Slytherin,Targaryen,Johnny Mnemonic
8,5/29/2018 18:42:59,Sandra Kruse,@Sandi_Peaches,English,CMU,"Pittsburgh, PA",Recent Masters Graduate [one year program],,2018-06-09,Late 19th and Early 20th Century American Lite...,MLA,I've done some readings and tutorials. Maybe I...,"Data Visualization, Databases, Blogging, Pedagogy",I'm unsure what role DH will/should/could play...,"I'm only vaguely familiar with the term, but, ...",I'm sorted Ravenclaw but friends claim I shoul...,Meh,My excuse for not having an answer is that I'm...
9,5/29/2018 20:23:10,Richard,@RLHeppner,Law,Duquesne University,"Pittsburgh, PA",Professor,,2018-06-10,"Law

In [ ]:
#First question: who is most likely to collaborate?
#how do we define that?
#places, years of program



In [108]:
def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

new = splitDataFrameList(data, 'DH_methods', ',')
final = splitDataFrameList(new, 'university', ',')

In [129]:
# final['DH_methods'] = final['DH_methods'].str.lower()
final['university'] = final['university'].str.lower()
# df = final[['DH_methods', 'university']]
# df.to_csv('navigator.csv', index=False)
df = final
# test = df.groupby(['first_name', 'university']).size().reset_index(name='weights')
T=nx.from_pandas_edgelist(df, 'first_name', 'university')
data = df.set_index('first_name').to_dict('index').items()
T.add_nodes_from(data)
degree = nx.degree(T)
clustering = nx.average_clustering(T)
partition = community.best_partition(T)
connected = nx.is_connected(T)
triangles = nx.triangles(T)
transitivity = nx.transitivity(T)
clusters = nx.clustering(T)
cliques = nx.find_cliques(T)
bip = bipartite.is_bipartite(T)
# bottom_nodes, top_nodes = bipartite.sets(T)
# all_nodes= bottom_nodes.union(top_nodes)
# color = bipartite.color(T)
# density = bipartite.density(T, bottom_nodes)
print( bip, connected, triangles, cliques, )
# test.columns = ['source', 'target', 'weights']
# test.to_csv('navigator.csv')

True False {'Zoe': 0, 'university of virginia': 0, 'Keanu': 0, 'ucla': 0, 'Golnar': 0, 'university of pittsburgh ': 0, 'Paul': 0, 'cmu': 0, 'Richard': 0, 'Taylor ': 0, 'duquesne university ': 0, 'Jack': 0, 'Lily ': 0, 'Sandra Kruse': 0} <generator object find_cliques at 0x10bab1938>


In [104]:
uni = bipartite.projected_graph(T, all_nodes)
print(uni.edges(data=True))

[(' machine learning/ai/buzzword tech stuff', ' text mining', {}), (' machine learning/ai/buzzword tech stuff', 'data visualization', {}), (' machine learning/ai/buzzword tech stuff', ' databases', {}), (' machine learning/ai/buzzword tech stuff', ' data visualization', {}), (' machine learning/ai/buzzword tech stuff', ' pedagogy', {}), (' machine learning/ai/buzzword tech stuff', 'networks', {}), (' machine learning/ai/buzzword tech stuff', ' website creation', {}), (' machine learning/ai/buzzword tech stuff', 'pedagogy', {}), (' machine learning/ai/buzzword tech stuff', ' blogging', {}), (' machine learning/ai/buzzword tech stuff', ' mapping', {}), (' text mining', 'data visualization', {}), (' text mining', ' databases', {}), (' text mining', ' data visualization', {}), (' text mining', ' pedagogy', {}), (' text mining', 'networks', {}), (' text mining', ' website creation', {}), (' text mining', ' blogging', {}), (' text mining', ' mapping', {}), ('data visualization', ' databases'

In [ ]:
#Second question: who is most likely to share interests?
#research interests, similar other interests

In [135]:
GOT = pd.read_csv('got.txt')
T=nx.from_pandas_edgelist(GOT, 'Source', 'Target')
data = GOT.set_index('Source').to_dict('index').items()
T.add_nodes_from(data)
degree = nx.degree(T)
clustering = nx.average_clustering(T)
partition = community.best_partition(T)
connected = nx.is_connected(T)
triangles = nx.triangles(T)
transitivity = nx.transitivity(T)
clusters = nx.clustering(T)
cliques = nx.find_cliques(T)
bip = bipartite.is_bipartite(T)
print( bip, connected, triangles, cliques, )

False True {'Aemon': 7, 'Grenn': 5, 'Samwell': 25, 'Aerys': 6, 'Jaime': 77, 'Robert': 44, 'Tyrion': 114, 'Tywin': 53, 'Alliser': 3, 'Mance': 21, 'Amory': 0, 'Oberyn': 9, 'Arya': 57, 'Anguy': 1, 'Beric': 9, 'Bran': 29, 'Brynden': 13, 'Cersei': 68, 'Gendry': 6, 'Gregor': 36, 'Joffrey': 60, 'Jon': 51, 'Rickon': 12, 'Roose': 4, 'Sandor': 39, 'Thoros': 8, 'Balon': 12, 'Loras': 18, 'Belwas': 3, 'Barristan': 8, 'Illyrio': 0, 'Hodor': 4, 'Jojen': 5, 'Luwin': 1, 'Meera': 7, 'Nan': 1, 'Theon': 5, 'Brienne': 13, 'Bronn': 4, 'Podrick': 5, 'Lothar': 8, 'Walder': 17, 'Catelyn': 45, 'Edmure': 15, 'Hoster': 2, 'Jeyne': 1, 'Lysa': 16, 'Petyr': 15, 'Robb': 72, 'Roslin': 5, 'Sansa': 82, 'Stannis': 32, 'Elia': 6, 'Ilyn': 15, 'Meryn': 19, 'Pycelle': 6, 'Shae': 6, 'Varys': 11, 'Craster': 6, 'Karl': 0, 'Daario': 5, 'Drogo': 5, 'Irri': 4, 'Daenerys': 14, 'Aegon': 0, 'Jorah': 8, 'Kraznys': 0, 'Missandei': 1, 'Rakharo': 0, 'Rhaegar': 6, 'Viserys': 1, 'Worm': 0, 'Davos': 1, 'Cressen': 0, 'Salladhor': 0, 'Eddard'